# Pretrain a neural network

In this tutorial, we demonstrate the steps to pretrain the Neural Network (NN) component of the hybrid model. The training data is derived from the pre-processed outputs generated after running the conceptual model. The NN is trained to predict key quantities - $ET$, $M$, $Pr$, $Ps$, and $Q$ - by matching the outputs of the conceptual model.

**IMPORTANT**: In this tutorial, we use a file listing 4 basins and demonstrate running a multi-basin model.

**Before we start**

- This tutorial is rendered from a Jupyter notebook that is hosted on GitHub. If you'd like to run the code yourself, you can access the notebook and configuration files directly from the repository: [02-PretrainNNmodel](https://github.com/jpcurbelo/torchHydroNodes/tree/master/tutorials/02-PretrainNNmodel).

- To run this notebook locally, ensure you have completed the setup steps outlined in [Getting started](https://torchhydronodes.readthedocs.io/en/latest/usage/getting_started.html). These steps include setting up the environment, installing the required packages, and preparing the data files necessary for the tutorial.

- **Dependency on a Previous Tutorial**: Before running this tutorial, you must complete the [01-RunConceptModel Tutorial](https://torchhydronodes.readthedocs.io/en/latest/tutorials/run-concept-model.html). After completing it:

    1- Move the generated run folder to ``src/data``.

    2- Update the ``data_dir`` field in the ``config_run_pretrain.yml``  ([here](https://github.com/jpcurbelo/torchHydroNodes/blob/master/tutorials/02-PretrainNNmodel/config_run_pretrain.yml)) file to point to this folder.

# Import packages

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Dynamically set the project directory based on the notebook's location
notebook_dir = Path().resolve()
project_dir = str(notebook_dir.parent.parent)  # Adjust based on your project structure
sys.path.append(project_dir)

from src.thn_run import (
    _load_cfg_and_ds,
    get_basin_interpolators
)

from src.modelzoo_concept import get_concept_model
from src.modelzoo_nn import (
    get_nn_model,
    get_nn_pretrainer,
)


# Constants

In [2]:
config_file = 'config_run_pretrain.yml'

# Load config file and prepare dataset

In [3]:
cfg, dataset = _load_cfg_and_ds(Path(config_file), model='pretrainer')

-- Loading the config file and the dataset
-- Using device: cpu --
Setting seed for reproducibility: 1111
-- Loading basin dynamics into xarray data set.
100%|██████████| 4/4 [00:00<00:00, 21.40it/s]


In [4]:
cfg._cfg['experiment_name']

'run_pretrain_nn_model'

A folder has been created in the ```runs``` directory with the name specified as ```experiment_name``` in the configuration file, appended with a ```YYMMDD_HHMMSS``` timestamp. This folder will contain the configuration, results, plots, and metrics associated with the run.

In [5]:
# Dataset attributes
dataset.__dict__.keys()

dict_keys(['cfg', 'is_train', '_compute_scaler', 'scaler', 'basins', '_disable_pbar', '_per_basin_target_stds', '_dates', 'start_and_end_dates', 'num_samples', 'period_starts', 'alias_map', 'alias_map_clean', 'ds_train', 'ds_valid', 'ds_static'])

In [6]:
display(
    'basins', dataset.basins,
    'start_and_end_dates', dataset.start_and_end_dates,
    'ds_train', dataset.ds_train,
    'ds_valid', dataset.ds_valid,
)

'basins'

['01013500', '01022500', '01030500', '06431500']

'start_and_end_dates'

{'train': {'start_date': Timestamp('1980-10-01 00:00:00'),
  'end_date': Timestamp('2000-09-30 00:00:00')},
 'valid': {'start_date': Timestamp('2000-10-01 00:00:00'),
  'end_date': Timestamp('2010-09-30 00:00:00')}}

'ds_train'

<xarray.Dataset> Size: 2MB
Dimensions:     (basin: 4, date: 7305)
Coordinates:
  * date        (date) datetime64[ns] 58kB 1980-10-01 1980-10-02 ... 2000-09-30
  * basin       (basin) <U8 128B '01013500' '01022500' '01030500' '06431500'
Data variables: (12/15)
    dayl        (basin, date) float32 117kB 11.33 11.28 11.23 ... 11.52 11.42
    et_bucket   (basin, date) float32 117kB 0.8673 1.153 1.251 ... 1.213 1.155
    m_bucket    (basin, date) float32 117kB 1.192e-07 1.192e-07 ... 0.1266
    obs_runoff  (basin, date) float32 117kB 0.551 0.5607 0.5586 ... 0.441 0.4353
    pr_bucket   (basin, date) float32 117kB 3.1 4.24 ... 1.192e-07 1.192e-07
    prcp        (basin, date) float32 117kB 3.1 4.24 8.02 15.27 ... 0.0 0.0 0.0
    ...          ...
    s_water     (basin, date) float32 117kB 1.303e+03 1.305e+03 ... 984.9 983.5
    srad        (basin, date) float32 117kB 192.6 206.3 165.4 ... 350.7 303.8
    tmax        (basin, date) float32 117kB 10.05 15.82 15.86 ... 22.11 19.83
    tmean       (basin, date) float32 117kB 6.08 10.53 11.84 ... 13.41 12.77
    tmin        (basin, date) float32 117kB 2.11 5.24 7.81 ... 1.95 4.71 5.72
    vp          (basin, date) float32 117kB 711.3 898.6 ... 755.6 868.9

'ds_valid'

<xarray.Dataset> Size: 906kB
Dimensions:     (basin: 4, date: 3652)
Coordinates:
  * date        (date) datetime64[ns] 29kB 2000-10-01 2000-10-02 ... 2010-09-30
  * basin       (basin) <U8 128B '01013500' '01022500' '01030500' '06431500'
Data variables: (12/15)
    dayl        (basin, date) float32 58kB 11.33 11.28 11.23 ... 11.52 11.52
    et_bucket   (basin, date) float32 58kB 1.485 1.449 1.463 ... 1.703 1.2 1.113
    m_bucket    (basin, date) float32 58kB 1.192e-07 1.192e-07 ... 0.1205 0.1076
    obs_runoff  (basin, date) float32 58kB 0.1158 0.1126 ... 0.4525 0.4525
    pr_bucket   (basin, date) float32 58kB 1.192e-07 1.192e-07 ... 1.192e-07
    prcp        (basin, date) float32 58kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...          ...
    s_water     (basin, date) float32 58kB 1.504e+03 1.502e+03 ... 1.047e+03
    srad        (basin, date) float32 58kB 327.8 331.3 314.3 ... 352.8 363.1
    tmax        (basin, date) float32 58kB 20.31 20.4 20.0 ... 26.98 20.26 19.66
    tmean       (basin, date) float32 58kB 12.23 11.92 12.17 ... 12.22 11.05
    tmin        (basin, date) float32 58kB 4.16 3.44 4.34 ... 8.61 4.18 2.44
    vp          (basin, date) float32 58kB 822.2 783.9 827.6 ... 810.6 710.9

Feel free to generate plots from the training and validation sets to get familiar with the data

# Create interpolators

As the time-series data was loaded on a one-day resolution, we need to run interpolation during the solution of the system of ODEs for adaptative-step methods and fixe-step methods with higher resolution.

In [7]:
# Get the basin interpolators
interpolators = get_basin_interpolators(dataset, cfg, project_dir)

# Create the Conceptual model

In [8]:
time_idx0 = 0 # Start from the first time index - 0 for training
model_concept = get_concept_model(cfg, dataset.ds_train, interpolators, 
                                  time_idx0, dataset.scaler)

# Create the Neural network model

In [9]:
model_nn = get_nn_model(model_concept, dataset.ds_static)

# Create the Pretrainer Model

In [10]:
pretrainer = get_nn_pretrainer(model_nn, dataset)

# Pretrain the model

In [11]:
pretrain_ok = pretrainer.train(loss=cfg.loss_pretrain, lr=cfg.lr_pretrain, 
                               epochs=cfg.epochs_pretrain,
                               disable_pbar=False,
                               any_log=False)

------------------------------------------------------------
-- Pretraining the neural network model -- (cpu)
------------------------------------------------------------
# Epoch 00001:   0%|          | 0/116 [00:00<?, ?it/s, Loss=2.1382e+00]

* Evaluating basin 06431500 (ds_valid): 100%|██████████| 4/4 [00:00<00:00, 46.46it/s]


Feel free to explore the *model_plots* and *model_results* folders to evaluate the outcomes of the pretraining stage. These folders provide insights into how well the model fits the target variables: $ET$, $M$, $Pr$, $Ps$, and $Q$.

The *model_weights* folder contains the best-performing version of the pretrained model, which will be loaded by the hybrid model at the start of the training process. This process will be demonstrated in the next tutorial: [03-TrainHybridModel](https://github.com/jpcurbelo/torchHydroNodes/tree/master/tutorials/03-TrainHybridModel).